### Step 1: Load the raw Excel file and inspect its structure

In this step, we load the Excel file **without headers** to understand the raw layout of the data.

### What we observe:
- The dataset does **not** follow a standard tabular format.
- The first few rows contain **metadata**, not actual records:
  - **Row 0** → Segment information (Consumer, Corporate, Home Office)
  - **Row 1** → Shipping modes (First Class, Same Day, etc.)
  - **Row 2** → Order ID label
- Actual sales data starts **from row 3 onward**.
- Many cells contain `NaN` values due to merged cells in the original Excel file.

### Why `header=None`?
- The file does not have a single clean header row.
- Reading it without headers allows us to manually interpret and restructure the data correctly in later steps.

The goal here is **inspection only** — no cleaning or transformation is done yet.


In [4]:
import pandas as pd

df = pd.read_excel("InputFiles/salesData.xlsx",header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Segment>>,Consumer,NaN,NaN,NaN,Consumer Total,Corporate,NaN,NaN,NaN,Corporate Total,Home Office,NaN,NaN,NaN,Home Office Total
1,Ship Mode>>,First Class,Same Day,Second Class,Standard Class,NaN,First Class,Same Day,Second Class,Standard Class,NaN,First Class,Same Day,Second Class,Standard Class,NaN
2,Order ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CA-2011-100293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.056,91.056
4,CA-2011-100706,NaN,NaN,129.44,NaN,129.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Step 2: Normalize headers and select valid columns

Customer segments and shipping modes are stored across multiple rows.
We extract these rows, apply forward fill to segment labels, and filter out non-data columns (Order ID, empty columns, and totals) to identify valid sales columns.


In [14]:
segment_row = df.loc[0]
shipmode_row = df.loc[1]

segment_row_filled = segment_row.ffill()



### Step 3: Build clean transactional rows

- Iterate over data rows (from row 3 onward)
- Skip summary rows (Grand / Total)
- Loop through valid sales columns
- Extract non-missing values
- Build normalized rows: order_id, segment, shipmode, sales


In [6]:
clean_columns = []

for col in df.columns:
    if col == 0:
        continue
    if pd.isna(shipmode_row[col]): # if this particular column is empty in shipment_row then skip it (isna means ismissing)
        continue
    if "Total" in str(segment_row_filled[col]):
        continue
    clean_columns.append(col)
print(clean_columns)

[1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14]


In [7]:
final_row = []

for rindex in range(3,len(df)):
    valueoftotaltoRemove = str(df.loc[rindex,0]).lower()
    if "total" in valueoftotaltoRemove or "grand" in valueoftotaltoRemove:
        continue
    order_id = df.loc[rindex,0]

    for col in clean_columns:
        value = df.loc[rindex,col]
        
        if pd.notna(value):
            final_row.append({
                "order_id":order_id,
                "segment":segment_row_filled[col],
                "shipmode":shipmode_row[col],
                "sales":value
            })

print(final_row)

[{'order_id': 'CA-2011-100293', 'segment': 'Home Office', 'shipmode': 'Standard Class', 'sales': 91.056}, {'order_id': 'CA-2011-100706', 'segment': 'Consumer', 'shipmode': 'Second Class', 'sales': 129.44}, {'order_id': 'CA-2011-100895', 'segment': 'Consumer', 'shipmode': 'Standard Class', 'sales': 605.47}, {'order_id': 'CA-2011-100916', 'segment': 'Corporate', 'shipmode': 'Standard Class', 'sales': 788.86}, {'order_id': 'CA-2011-101266', 'segment': 'Consumer', 'shipmode': 'Second Class', 'sales': 13.36}, {'order_id': 'CA-2011-101560', 'segment': 'Corporate', 'shipmode': 'Second Class', 'sales': 542.3399999999999}, {'order_id': 'CA-2011-101770', 'segment': 'Corporate', 'shipmode': 'Standard Class', 'sales': 1.8690000000000004}, {'order_id': 'CA-2011-102274', 'segment': 'Corporate', 'shipmode': 'Standard Class', 'sales': 865.5}, {'order_id': 'CA-2011-102673', 'segment': 'Corporate', 'shipmode': 'Standard Class', 'sales': 1044.4399999999998}, {'order_id': 'CA-2011-102988', 'segment': 'Cor

### Step 4: Export cleaned data to CSV

In [8]:
finalData = pd.DataFrame(final_row)
finalData.tail()

#finalData.to_csv("output/copied.csv")

,order_id,segment,shipmode,sales
817,US-2014-166233,Consumer,Standard Class,24.000
818,US-2014-166611,Corporate,Standard Class,68.742
819,US-2014-167920,Consumer,Second Class,1827.510
820,US-2014-168116,Corporate,Same Day,8167.420
821,US-2014-168690,Consumer,Standard Class,2.808


In [9]:
total_sales = finalData["sales"].sum()
print(total_sales)

391721.905


In [10]:
total_Sales_Corporate = finalData[finalData["segment"] == "Corporate"]["sales"].sum()
print(total_Sales_Corporate)

121885.93250000001


In [11]:
get_shipmode_sales = finalData[["shipmode","sales"]]
get_shipmode_sales.head()

,shipmode,sales
0,Standard Class,91.056
1,Second Class,129.440
2,Standard Class,605.470
3,Standard Class,788.860
4,Second Class,13.360


In [12]:
Shipmode_names = finalData["shipmode"].unique()
print(Shipmode_names)


['Standard Class' 'Second Class' 'First Class' 'Same Day']


### Step 5: Analyze sales by shipping mode

- Aggregate total sales for each shipping mode
- Create a summary DataFrame
- Calculate percentage contribution of each shipping mode
- Format percentages for readable output

In [13]:
shipmode_wise_totalSales = {}
for i in Shipmode_names:
    getsum = finalData[finalData["shipmode"] == i]["sales"].sum()
    shipmode_wise_totalSales[i] = getsum.item()
    
shipmode_Df = pd.DataFrame(shipmode_wise_totalSales.items(),columns=["Ship_mode","Total_sales"])
totalSales= shipmode_Df["Total_sales"].sum()
shipmode_Df["Percentage"] = ((shipmode_Df["Total_sales"]/totalSales)*100)
shipmode_Df["Percentage"] = shipmode_Df["Percentage"].round(1).astype(str) + "%"
shipmode_Df.head()

,Ship_mode,Total_sales,Percentage
0,Standard Class,227613.5535,58.1%
1,Second Class,93758.6125,23.9%
2,First Class,49332.5660,12.6%
3,Same Day,21017.1730,5.4%
